In [ ]:
!pip install transformers --quiet
!pip install torch  --quiet

Final Task: Aim to classify the tweets.

In [ ]:
import numpy as np
import pandas as pd
import transformers
from transformers import pipeline

In [ ]:
!gdown --id 1RARLRRVmkaoHQV05pfGIvyBH6XRjcSr9

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RARLRRVmkaoHQV05pfGIvyBH6XRjcSr9
To: /content/tweets.csv
100% 8.70M/8.70M [00:00<00:00, 197MB/s]


In [ ]:
data= pd.read_csv('tweets.csv')
data

,Unnamed: 0,tweet,target
0,0,ndtv anchor let go over to aap live press conf...,0
1,1,sambit patra attacks rahul gandhi over his cow...,0
2,2,crediting the bjp with taking development to s...,1
3,3,ktrbrs do do false allegations on iasassociati...,0
4,4,gujarat assembly yet to get rs cr as gst compe...,1
...,...,...,...
49472,25683,1ddrive modispeakstonews18 loksabhaelections20...,1
49473,25684,i am seriously worried about future of indian ...,1
49474,25685,if narendramodi is so bad then why does incind...,0
49475,25686,on this 23rd march pakistan day i urge all ind...,0


Government tweets are labeled with 1 while opposition tweets are labeled with 0. I have already done the data pre-processing part to make it easy for you all.
Go-ahead and do **encode -> spliting -> classification using model**.
You have already done this task in last two assignments. **Best of Luck!**

Optional: You can also try clustring algorithem, this would give you a better result.

In [ ]:
from transformers import AutoTokenizer
import torch


# Preprocess the data
data = data.dropna()
data['tweet'] = data['tweet'].str.lower()

# Encode the data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoded_data = tokenizer.batch_encode_plus(
    data['tweet'].tolist(),
    padding='longest',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Extract the input IDs and labels
input_ids = encoded_data['input_ids']
labels = torch.tensor(data['target'].tolist())

# Print the encoded data
print('Input IDs shape:', input_ids.shape)
print('Labels shape:', labels.shape)


Input IDs shape: torch.Size([49477, 128])
Labels shape: torch.Size([49477])


In [ ]:
for i in range(5):
    print("Tweet:", data['tweet'][i])
    print("Input IDs:", encoded_data['input_ids'][i])
    print("Attention Mask:", encoded_data['attention_mask'][i])
    print("Label:", labels[i])
    print()

Tweet: ndtv anchor let go over to aap live press conference the next thing you see sanjayazadsln bashing bjp with modiadanibhaibhai probably an awkward moment for everyone in the ndtv news room ca even stop the live stream midway manishsisodia bjp aap
Input IDs: tensor([  101,  1050, 11927,  2615,  8133,  2292,  2175,  2058,  2000,  9779,
         2361,  2444,  2811,  3034,  1996,  2279,  2518,  2017,  2156, 29590,
        10936, 19303, 19666, 24234,  2075, 24954,  2007, 16913, 28665,  7088,
        22655, 12322, 10932,  2763,  2019,  9596,  2617,  2005,  3071,  1999,
         1996,  1050, 11927,  2615,  2739,  2282,  6187,  2130,  2644,  1996,
         2444,  5460, 12213, 23624,  4095,  6190,  7716,  2401, 24954,  9779,
         2361,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
train_input_ids, test_input_ids, train_labels, test_labels = train_test_split(
    input_ids,
    labels,
    test_size=0.2,
    random_state=42
)

# Print the sizes of train and test sets
print('Train Set Size:', len(train_input_ids))
print('Test Set Size:', len(test_input_ids))
print()

Train Set Size: 39581
Test Set Size: 9896



In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification
# Create the transformer model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Print the model architecture
print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
pip install transformers[torch]

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
batch_size = 8

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for i in range(0, len(train_input_ids), batch_size):
        inputs = train_input_ids[i:i+batch_size].to(device)
        labels = train_labels[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Compute average training loss
    train_loss /= len(train_input_ids) / batch_size

    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {train_loss:.4f}")

KeyboardInterrupt: ignored

In [ ]:
model.eval()  # Set the model to evaluation mode
eval_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for i in range(0, len(test_input_ids), batch_size):
        inputs = test_input_ids[i:i+batch_size].to(device)
        labels = test_labels[i:i+batch_size].to(device)

        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        eval_loss += loss.item()

        _, predicted = torch.max(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    eval_loss /= len(test_input_ids) / batch_size
    accuracy = correct / total * 100

print(f"Evaluation Loss: {eval_loss:.4f}")
print(f"Accuracy: {accuracy:.2f}%")
